In [1]:
from pathlib import Path
import torch
from ray.rllib.models.torch.torch_distributions import TorchDiagGaussian
from ray.tune.result import TRAINING_ITERATION
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.core import (
    COMPONENT_LEARNER,
    COMPONENT_LEARNER_GROUP,
    COMPONENT_RL_MODULE,
)
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.examples.envs.classes.multi_agent import MultiAgentPendulum
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
    NUM_ENV_STEPS_SAMPLED_LIFETIME,
)
from ray.rllib.utils.numpy import convert_to_numpy
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    check,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env
import os

import gymnasium as gym
import numpy as np
import tree  # pip install dm_tree

from ray.rllib.core import DEFAULT_MODULE_ID
from ray.rllib.core.columns import Columns
from ray.rllib.core.rl_module.rl_module import RLModule
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.numpy import convert_to_numpy, softmax
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
)
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls
import os

import gymnasium as gym
import numpy as np
import tree  # pip install dm_tree

from ray.rllib.core import DEFAULT_MODULE_ID
from ray.rllib.core.columns import Columns
from ray.rllib.core.rl_module.rl_module import RLModule
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.numpy import convert_to_numpy, softmax
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
)
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls


/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-16 19:26:47,829	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-16 19:26:48,076	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:

torch, _ = try_import_torch()

parser = add_rllib_example_script_args(default_reward=200.0)
parser.add_argument(
    "--use-onnx-for-inference",
    action="store_true",
    help="Whether to convert the loaded module to ONNX format and then perform "
    "inference through this ONNX model.",
)
parser.add_argument(
    "--explore-during-inference",
    action="store_true",
    help="Whether the trained policy should use exploration during action "
    "inference.",
)
parser.add_argument(
    "--num-episodes-during-inference",
    type=int,
    default=10,
    help="Number of episodes to do inference over (after restoring from a checkpoint).",
)
parser.set_defaults(
    # Make sure that - by default - we produce checkpoints during training.
    checkpoint_freq=1,
    env="waterworld_v4",
    checkpoint_at_end=True,
    # Use CartPole-v1 by default.
    # Script only runs on new API stack.
    enable_new_api_stack=True,
)

In [3]:
import sys
sys.argv = [
    'notebook_script.py',
    '--enable-new-api-stack',
    '--num-agents=4',
    # '--no-tune',
    '--checkpoint-at-end',
    '--stop-reward=200.0',

]
print("参数设置完成")

参数设置完成


In [4]:
parser = add_rllib_example_script_args(
    default_iters=2,
    default_timesteps=10000,
    default_reward=0.0,
)

args = parser.parse_args()

assert args.num_agents > 0, "Must set --num-agents > 0 when running this script!"
assert (
    args.enable_new_api_stack
), "Must set --enable-new-api-stack when running this script!"

print(f"参数解析完成: num_agents={args.num_agents}, algo={args.algo}")


参数解析完成: num_agents=4, algo=PPO


In [5]:
# Cell 4: 注册环境和配置算法
# Here, we use the "Agent Environment Cycle" (AEC) PettingZoo environment type.
# For a "Parallel" environment example, see the rock paper scissors examples
# in this same repository folder.
# Here, we use the "Agent Environment Cycle" (AEC) PettingZoo environment type.
# For a "Parallel" environment example, see the rock paper scissors examples
# in this same repository folder.
register_env("env", lambda _: PettingZooEnv(waterworld_v4.env()))

# Policies are called just like the agents (exact 1:1 mapping).
policies = {f"pursuer_{i}" for i in range(args.num_agents)}
print(policies)

{'pursuer_1', 'pursuer_2', 'pursuer_3', 'pursuer_0'}


In [6]:

base_config = (
    get_trainable_cls(args.algo)
    .get_default_config()
    .environment("env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .training(
        vf_loss_coeff=0.005,
    )
    .rl_module(
        rl_module_spec=MultiRLModuleSpec(
            rl_module_specs={p: RLModuleSpec() for p in policies},
        ),
        model_config=DefaultModelConfig(vf_share_layers=True),
    )
)



/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [7]:
# run_rllib_example_script_experiment(base_config, args)
results = run_rllib_example_script_experiment(base_config, args, keep_ray_up=True)
print("环境注册和配置完成")

KeyboardInterrupt: 

In [12]:
args.use_onnx_for_inference=False
args.explore_during_inference=False
if args.use_onnx_for_inference:
    if args.explore_during_inference:
        raise ValueError(
            "Can't set `--explore-during-inference` and `--use-onnx-for-inference` "
            "together! ONNX models use the original RLModule's `forward_inference` "
            "only."
        )
    import onnxruntime

    base_config = get_trainable_cls(args.algo).get_default_config()

In [13]:
print("Training completed. Restoring new RLModule for action inference.")
# Get the last checkpoint from the above training run.
best_result = results.get_best_result(
    metric=f"{ENV_RUNNER_RESULTS}/{EPISODE_RETURN_MEAN}", mode="max"
)

Training completed. Restoring new RLModule for action inference.


In [14]:
rl_module0 = RLModule.from_checkpoint(
    os.path.join(
        best_result.checkpoint.path,
        "learner_group",
        "learner",
        "rl_module",
        "pursuer_0" ,
    )
)
rl_module1 = RLModule.from_checkpoint(
    os.path.join(
        best_result.checkpoint.path,
        "learner_group",
        "learner",
        "rl_module",
        "pursuer_1" ,
    )
)

2025-06-26 07:17:54,360	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!



DefaultPPOTorchRLModule(
  (encoder): TorchActorCriticEncoder(
    (encoder): TorchMLPEncoder(
      (net): TorchMLP(
        (mlp): Sequential(
          (0): Linear(in_features=242, out_features=256, bias=True)
          (1): Tanh()
          (2): Linear(in_features=256, out_features=256, bias=True)
          (3): Tanh()
        )
      )
    )
  )
  (pi): TorchMLPHead(
    (net): TorchMLP(
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=4, bias=True)
      )
    )
  )
  (vf): TorchMLPHead(
    (net): TorchMLP(
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=True)
      )
    )
  )
)

In [15]:
ort_session = None
print(" ok")

 ok


In [16]:
# num_episodes = 0
# base_config.explore = False
# algo = base_config.build()
# checkpoint = results.get_best_result().checkpoint
# algo.restore(checkpoint)
# rl_module0 = algo.get_module("pursuer_1")
# print(rl_module0)



In [17]:
env = waterworld_v4.env(render_mode="human",n_pursuers=4)
env.reset(seed=42)

for agent in env.agent_iter():
    print(agent)
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        # action = env.action_space(agent).sample()


        # Compute an action using a B=1 observation "batch".
        input_dict = {Columns.OBS: np.expand_dims(observation, 0)}
        if not args.use_onnx_for_inference:
            input_dict = {Columns.OBS: torch.from_numpy(observation).unsqueeze(0)}

        # If ONNX and module has not been exported yet, do this here using
        # the input_dict as example input.
        elif ort_session is None:
            tensor_input_dict = {Columns.OBS: torch.from_numpy(observation).unsqueeze(0)}
            torch.onnx.export(rl_module, {"batch": tensor_input_dict}, f="test.onnx")
            ort_session = onnxruntime.InferenceSession(
                "test.onnx", providers=["CPUExecutionProvider"]
            )

        # No exploration (using ONNX).
        if ort_session is not None:
            rl_module_out = ort_session.run(
                None,
                {
                    key.name: val
                    for key, val in dict(
                        zip(
                            tree.flatten(ort_session.get_inputs()),
                            tree.flatten(input_dict),
                        )
                    ).items()
                },
            )
            # [0]=encoder outs; [1]=action logits
            rl_module_out = {Columns.ACTION_DIST_INPUTS: rl_module_out[1]}
        # No exploration (using RLModule).
        elif not args.explore_during_inference:
            rl_module_out = rl_module0.forward_inference(input_dict)["action_dist_inputs"][0].numpy()
            action = TorchDiagGaussian.from_logits(torch.tensor(rl_module_out).unsqueeze(0)).sample().squeeze(0).numpy()

        # W/ exploration (using RLModule).
        else:
            rl_module_out = rl_module1.forward_inference(input_dict)["action_dist_inputs"][0].numpy()
            action = TorchDiagGaussian.from_logits(torch.tensor(rl_module_out).unsqueeze(0)).sample().squeeze(0).numpy()

    env.step(action)
env.close()

predator_0
predator_1
[WARNING]: Received an action [ 0.2015424 -1.8714719] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
prey_0
prey_1
[WARNING]: Received an action [-1.5084137   0.33833247] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
predator_0
[WARNING]: Received an action [ 1.379974  -2.1793265] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
predator_1
prey_0
[WARNING]: Received an action [-0.77383125 -1.6188028 ] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
prey_1
predator_0
predator_1
[WARNING]: Received an action [-0.39944845 -1.3716171 ] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to space
prey_0
prey_1
[WARNING]: Received an action [0.29721907 1.006066  ] that was outside action space Box(-1.0, 1.0, (2,), float32). Environment is clipping to spac

In [18]:
import sys
sys.argv = [
    'notebook_script.py',
    '--enable-new-api-stack',
    '--num-agents=2',
    '--checkpoint-freq=20',

    '--checkpoint-at-end',
    '--stop-reward=200.0',

]
print("参数设置完成")

参数设置完成
